![image](resources/qcqh-lab-header.png)

## Lab 8: Quantum Chemistry

<img src="resources/quantum-chemistry.png">

In this lab, you will run through an example of a quantum chemistry problem. You will be exploring the simulation of the molecule LiH in Qiskit. You will need to follow the video associated with this lab.

### Installing necessary packages

Before we begin, you will need to install some prerequisites into your environment. Run the cell below to complete these installations. At the end, the cell outputs will be cleared.

In [1]:
!pip install -U -r resources/requirements.txt

from IPython.display import clear_output
clear_output()
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Simulating LiH

In [2]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

molecular_coordinates = 'Li 0 0 0; H 1.6 0 0'

distance_unit = UnitsType.ANGSTROM

basis = 'sto3g'

driver = PySCFDriver(molecular_coordinates, unit=distance_unit)

molecule = driver.run()

h1 = molecule.one_body_integrals

print(h1)

ImportError: cannot import name 'BaseBackend' from 'qiskit.providers' (/home/mgabijo/.local/lib/python3.8/site-packages/qiskit/providers/__init__.py)

In [3]:
h2 = molecule.two_body_integrals
print(h2)

[[[[-8.29283343e-01  5.58549882e-02  6.92873148e-02 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [ 5.58549882e-02 -6.66878909e-03 -5.60788725e-03 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [ 6.92873148e-02 -5.60788725e-03 -1.08311226e-02 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   ...
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]]

  [[ 5.58549882e-02 -6.66878909e-03 -5.60788725e-03 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [-1.83350520e-01 -3.10515039e-03  7.93404250e-03 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   [-6.72563783e-03  1.67469536e-03 -8.81386236e-05 ...  0.00000000e+00
     0.00000000e+00  0.00000000e+00]
   ...
   

In [4]:
nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

print(nuclear_repulsion_energy)

0.992207270475


# 1. Creating a Fermionic Operator 

In [5]:
from qiskit.chemistry import FermionicOperator

ferOp = FermionicOperator(h1=h1, h2=h2)

num_spin_orbitals = molecule.num_orbitals * 2
num_particles = molecule.num_alpha + molecule.num_beta

freeze_list = [0,6] 

remove_list = [3,8]


In [6]:
ferOp_f, energy_shift = ferOp.fermion_mode_freezing(freeze_list)

num_spin_orbitals -= len(freeze_list)
num_particles -= len(freeze_list)

print(ferOp_f.h1)

[[-0.77258172  0.0485796   0.          0.         -0.12679498  0.
   0.          0.          0.          0.        ]
 [ 0.0485796  -0.35593954  0.          0.          0.06813315  0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -0.35297897  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.35297897  0.          0.
   0.          0.          0.          0.        ]
 [-0.12679498  0.06813315  0.          0.         -0.236054    0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.77258172
   0.0485796   0.          0.         -0.12679498]
 [ 0.          0.          0.          0.          0.          0.0485796
  -0.35593954  0.          0.          0.06813315]
 [ 0.          0.          0.          0.          0.          0.
   0.         -0.35297897  0.          0.        ]
 [ 0.          0.          0.          0.        

In [7]:
ferOp_fr = ferOp_f.fermion_mode_elimination(remove_list)

num_spin_orbitals -= len(remove_list)

print(ferOp_fr.h1)

[[-0.77258172  0.0485796   0.         -0.12679498  0.          0.
   0.          0.        ]
 [ 0.0485796  -0.35593954  0.          0.06813315  0.          0.
   0.          0.        ]
 [ 0.          0.         -0.35297897  0.          0.          0.
   0.          0.        ]
 [-0.12679498  0.06813315  0.         -0.236054    0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.77258172  0.0485796
   0.         -0.12679498]
 [ 0.          0.          0.          0.          0.0485796  -0.35593954
   0.          0.06813315]
 [ 0.          0.          0.          0.          0.          0.
  -0.35297897  0.        ]
 [ 0.          0.          0.          0.         -0.12679498  0.06813315
   0.         -0.236054  ]]


# 2. Creating a Qubit Operator 

In [ ]:
map_type = 'parity'

qubitOp = ferOp_fr.mapping(map_type=map_type)

print(qubitOp)
print(qubitOp.print_details())

In [ ]:
from qiskit.aqua.operators import Z2Symmetries

pauli_symm = Z2Symmetries.find_Z2_symmetries(qubitOp)
print(pauli_symm)

In [ ]:
qubitOp_t = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)

print(num_particles)

print(qubitOp_t)
print(qubitOp_t.print_details())

In [ ]:
from qiskit.aqua.algorithms import NumPyEigensolver

ee = NumPyEigensolver(qubitOp_t)

result = ee.run()
ref = result['eigenvalues']

print(ref + nuclear_repulsion_energy + energy_shift)

# Creating a Variational Ansatz 

In [ ]:
from qiskit.chemistry.components.initial_states import HartreeFock

init_state = HartreeFock(num_orbitals=4, num_particles=2, qubit_mapping='jordan_wigner')

print(init_state.bitstr)

HF_circuit = init_state.construct_circuit('circuit')
HF_circuit.decompose().draw(output='mpl')

In [ ]:
from qiskit.chemistry.components.variational_forms import UCCSD

UCCSD_var_form = UCCSD(num_orbitals=4, num_particles=2, qubit_mapping='jordan_wigner',
                      excitation_type='s', method_singles='beta',
                      initial_state=init_state, two_qubit_reduction=False, reps=2)

print(UCCSD_var_form.single_excitations)
print(UCCSD_var_form.num_parameters)

In [ ]:
var_circuit = UCCSD_var_form.construct_circuit([1,2])

var_circuit.decompose().draw(output='mpl')

# Homework - The last symmetry standing 

In [ ]:
pauli_symm = Z2Symmetries.find_Z2_symmetries(qubitOp_t)
print(pauli_symm)